<a href="https://colab.research.google.com/github/gimalgosa0514/SW/blob/main/DeepFashion_Try_On_Look_For_You.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepFashion Try-On

Towards Photo-Realistic Virtual Try-On by Adaptively Generating↔Preserving Image Content, CVPR'20.

![](https://github.com/switchablenorms/DeepFashion_Try_On/raw/master/images/tryon.png)

## For inferencing ACGPN!

ACGPN repo: https://github.com/switchablenorms/DeepFashion_Try_On

This notebook is hard coded for inferencing one image at a time.

Notebook by [Levin Dabhi](https://levindabhi.github.io/)

```
author = {Yang, Han and Zhang, Ruimao and Guo, Xiaobao and Liu, Wei and Zuo, Wangmeng and Luo, Ping},
title = {Towards Photo-Realistic Virtual Try-On by Adaptively Generating-Preserving Image Content},
booktitle = {IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR)},
month = {June},
year = {2020}
}

@inproceedings{ge2021disentangled,
  title={Disentangled Cycle Consistency for Highly-realistic Virtual Try-On},
  author={Ge, Chongjian and Song, Yibing and Ge, Yuying and Yang, Han and Liu, Wei and Luo, Ping},
  booktitle={Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition},
  pages={16928--16937},
  year={2021}
}

@inproceedings{yang2022full,
title = {Full-Range Virtual Try-On With Recurrent Tri-Level Transform},
author = {Yang, Han and Yu, Xinrui and Liu, Ziwei},
booktitle = {Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition},
pages = {3460--3469}
year = {2022}
}
```

## ACGPN

- Original: https://github.com/levindabhi/ACGPN.git
- Modified: https://github.com/kairess/ACGPN.git

In [ ]:
!git clone https://github.com/kairess/ACGPN.git    #git을 이용해서 저 URL에 있는 코드를 내려받아서 로컬 환경으로 복제한다.
%cd ACGPN     
#현재 작업 디렉토리를 ACGPN 디렉토리로 바꾼다. 여기서 ACGPN은 누끼 딴 옷 이미지와 사람 이미지를 합성하는 모델로 추정됨

Cloning into 'ACGPN'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 165 (delta 21), reused 19 (delta 18), pack-reused 141
Receiving objects: 100% (165/165), 303.15 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (62/62), done.
/content/ACGPN


## Dependencies

In [ ]:
!pip install -U --no-cache-dir gdown --pre -qq     #gdown이라는 Python 패키지를 설치하는 명령어입니다. gdown패키지는 구글 드라이브에서 파일을 다운로드하는데 사용된다.
#--no-cache-dir 옵션은 패키지 캐시를 사용하지 않도록 지정합니다. 
#--pre 옵션은 미리 릴리스 된 패키지뿐만 아니라 베타 또는 릴리스 전 버전의 패키지도 설치할 수 있도록 해줍니다
#-qq 옵션은 출력 메시지를 최소화하므로 설치 과정에서 출력되는 메시지가 적어집니다.

!pip install ninja -qq   #ninja라는 빌드 시스템을 설치하는 명령어입니다. 
#ninja는 C++ 등의 프로그래밍 언어로 작성된 프로젝트에서 빌드 시스템으로 사용되는 빌드 도구입니다.
#ninja는 build.ninja 라는 파일 형식을 사용하여 빌드 과정을 기술합니다. 
#이 파일에는 각 빌드 타겟과 해당 타겟이 의존하는 파일, 그리고 빌드 명령어 등이 기술되어 있습니다. 
#이렇게 작성된 build.ninja 파일은 ninja 명령어를 실행하여 빌드를 수행할 수 있습니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 4.2 MB/s eta 0:00:00


In [ ]:
import gdown #gdown 라이브러리를 사용하여 Google Drive에 저장된 모델 가중치 파일을 다운로드
import numpy as np #NumPy는 파이썬에서 사용되는 대규모 다차원 배열 및 행렬 연산을 위한 라이브러리
from PIL import Image #Image 모듈은 이미지 파일을 열고 읽어들이고, 크기를 조절하고, 필터를 적용하고, 이미지를 저장하는 등의 다양한 작업을 수행
import IPython
import gdown
import os #운영체제와 상호작용
import sys 
import time

from predict_pose import generate_pose_keypoints 
#generate_pose_keypoints 함수를 사용하여 이미지에서 사람의 포즈를 추론하고, 추론된 포즈의 키포인트 좌표를 반환합니다.

In [ ]:
!mkdir Data_preprocessing/test_color
!mkdir Data_preprocessing/test_colormask
!mkdir Data_preprocessing/test_edge
!mkdir Data_preprocessing/test_img
!mkdir Data_preprocessing/test_label
!mkdir Data_preprocessing/test_mask
!mkdir Data_preprocessing/test_pose
!mkdir inputs
!mkdir inputs/img
!mkdir inputs/cloth

#디렉토리 생성

In [ ]:
!git clone https://github.com/levindabhi/Self-Correction-Human-Parsing-for-ACGPN.git  #사람의 포즈를 예측하고 상의,하의,악세사리등을 인식할 수 있는 모델
!git clone https://github.com/levindabhi/U-2-Net.git #옷 누끼 따는 모델

Cloning into 'Self-Correction-Human-Parsing-for-ACGPN'...
remote: Enumerating objects: 769, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 769 (delta 72), reused 62 (delta 62), pack-reused 658
Receiving objects: 100% (769/769), 3.80 MiB | 17.68 MiB/s, done.
Resolving deltas: 100% (189/189), done.
Cloning into 'U-2-Net'...
remote: Enumerating objects: 822, done.
remote: Counting objects: 100% (822/822), done.
remote: Compressing objects: 100% (439/439), done.
remote: Total 822 (delta 389), reused 793 (delta 379), pack-reused 0
Receiving objects: 100% (822/822), 30.71 MiB | 40.11 MiB/s, done.
Resolving deltas: 100% (389/389), done.


## 사전학습 모델 다운로드

### 포즈 예측 모델

In [ ]:
!gdown 1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko -O pose/pose_iter_440000.caffemodel
#1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko ID를 가진 파일이 pose/pose_iter_440000.caffemodel 경로에 다운로드됩니다. 이 명령어를 실행하기 위해서는 gdown 패키지가 미리 설치되어 있어야 합니다.
#포즈 예측 모델인듯

Downloading...
From (uriginal): https://drive.google.com/uc?id=1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko
From (redirected): https://drive.google.com/uc?id=1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko&confirm=t&uuid=0c157d22-f8c1-4083-aa1b-0ddf64f90924
To: /content/ACGPN/pose/pose_iter_440000.caffemodel
100% 209M/209M [00:08<00:00, 24.3MB/s]


### 휴먼 세그멘테이션 마스크 생성 모델

In [ ]:
gdown.download('https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH', 'lip_final.pth', quiet=False)
#저 url 주소에 있는 파일을 다운로드해서 이름을 lip_final.pth로 저장한다. quiet를 false로 지정하면 다운로드 진행상황을 알려준다.
#gdown은 google drive에 대용량 파일을 다운로드 할 수 있게 한다.

Downloading...
From (uriginal): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
From (redirected): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH&confirm=t&uuid=6820da67-b4a9-419b-8ab2-32fd4e9dc749
To: /content/ACGPN/lip_final.pth
100%|██████████| 267M/267M [00:10<00:00, 25.0MB/s]


'lip_final.pth'

### U2Net 모델

옷 마스크 추출 모델

In [ ]:
%cd U-2-Net
!mkdir saved_models
!mkdir saved_models/u2net
!mkdir saved_models/u2netp

!gdown 1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy -O saved_models/u2netp/u2netp.pth
!gdown 1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ -O saved_models/u2net/u2net.pth

import u2net_load #u2net_load 모듈은 U-2-Net 모델을 로드하고 초기화하는 함수를 제공합니다. 
import u2net_run #u2net_run 모듈은 입력 이미지를 U-2-Net 모델에 전달하여 전경과 배경을 분리하는 함수를 제공합니다. 

u2net = u2net_load.model(model_name='u2netp') #U2Net 모델 불러오기

%cd .. #이전 작업 폴더로 돌아감

/content/ACGPN/U-2-Net
mkdir: cannot create directory ‘saved_models’: File exists
Downloading...
From: https://drive.google.com/uc?id=1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy
To: /content/ACGPN/U-2-Net/saved_models/u2netp/u2netp.pth
100% 4.68M/4.68M [00:00<00:00, 203MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ
From (redirected): https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ&confirm=t&uuid=50508f84-6c84-455a-b30f-bd762a84cb98
To: /content/ACGPN/U-2-Net/saved_models/u2net/u2net.pth
100% 176M/176M [00:00<00:00, 199MB/s]
...load U2NEP---4.7 MB
[Errno 2] No such file or directory: '.. #이전 작업 폴더로 돌아감'
/content/ACGPN/U-2-Net


### ACGPN 모델

In [ ]:
!mkdir checkpoints

gdown.download('https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx', output='checkpoints/ACGPN_checkpoints.zip', quiet=False) #저 url 파일을 다운받아서 checkpoints 디렉토리에 ACGPN_checkopoints.zip으로 저장한다.

!unzip checkpoints/ACGPN_checkpoints.zip -d checkpoints #ACGPN_checkpoints.zip을 압축해제한다.
#이 작업은 ACGPN 모델의 학습된 가중치를 다운로드하고, 모델을 사용하여 이미지를 처리할 수 있도록 하는 데 필요합니다.

Downloading...
From (uriginal): https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx
From (redirected): https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx&confirm=t&uuid=15ccca49-84aa-4ef1-9364-1de65dddde5e
To: /content/ACGPN/U-2-Net/checkpoints/ACGPN_checkpoints.zip
100%|██████████| 524M/524M [00:02<00:00, 224MB/s]


Archive:  checkpoints/ACGPN_checkpoints.zip
   creating: checkpoints/label2city/
  inflating: checkpoints/label2city/latest_net_G.pth  
  inflating: checkpoints/label2city/latest_net_G1.pth  
  inflating: checkpoints/label2city/latest_net_G2.pth  
  inflating: checkpoints/label2city/latest_net_U.pth  
  inflating: checkpoints/label2city/opt.txt  


## VITON 데이터셋

https://drive.google.com/uc?id=1tE7hcVFm8Td8kRh5iYRBSDFdvZIkbUIR

## AI허브 패션 데이터셋

https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=78

![](https://aihub.or.kr/web-nas/aihub21/files/public/inline-images/65_%ED%8C%A8%EC%85%98%EC%83%81%ED%92%88%EB%B0%8F%EC%B0%A9%EC%9A%A9%EC%98%81%EC%83%81_%EB%8C%80%ED%91%9C%EB%8F%84.PNG)

## 전처리

### 옷 마스크 추출

In [ ]:
sorted(os.listdir('inputs/cloth'))

FileNotFoundError: ignored

In [ ]:
cloth_name = f'cloth_{int(time.time())}.png'

cloth_path = os.path.join('inputs/cloth', sorted(os.listdir('inputs/cloth'))[0])
cloth = Image.open(cloth_path)
cloth = cloth.resize((192, 256), Image.BICUBIC).convert('RGB')
cloth.save(os.path.join('Data_preprocessing/test_color', cloth_name))

u2net_run.infer(u2net, 'Data_preprocessing/test_color', 'Data_preprocessing/test_edge')

Image.open(f'Data_preprocessing/test_edge/{cloth_name}')

FileNotFoundError: ignored

### 모델 포즈, 세그멘테이션

In [ ]:
sorted(os.listdir('inputs/img'))

FileNotFoundError: ignored

In [ ]:
img_name = f'img_{int(time.time())}.png'

img_path = os.path.join('inputs/img', sorted(os.listdir('inputs/img'))[0])
img = Image.open(img_path)
img = img.resize((192,256), Image.BICUBIC)

img_path = os.path.join('Data_preprocessing/test_img', img_name)
img.save(img_path)

!python3 Self-Correction-Human-Parsing-for-ACGPN/simple_extractor.py --dataset 'lip' --model-restore 'lip_final.pth' --input-dir 'Data_preprocessing/test_img' --output-dir 'Data_preprocessing/test_label'

pose_path = os.path.join('Data_preprocessing/test_pose', img_name.replace('.png', '_keypoints.json'))
generate_pose_keypoints(img_path, pose_path)

FileNotFoundError: ignored

## 추론

In [ ]:
!rm -rf Data_preprocessing/test_pairs.txt
with open('Data_preprocessing/test_pairs.txt', 'w') as f:
    f.write(f'{img_name} {cloth_name}')

FileNotFoundError: ignored

In [ ]:
!python test.py

python3: can't open file '/content/ACGPN/U-2-Net/ACGPN/U-2-Net/test.py': [Errno 2] No such file or directory


## 결과

In [ ]:
output_grid = np.concatenate([
    np.array(Image.open(f'Data_preprocessing/test_img/{img_name}')),
    np.array(Image.open(f'Data_preprocessing/test_color/{cloth_name}')),
    np.array(Image.open(f'results/test/try-on/{img_name}'))
], axis=1)

image_grid = Image.fromarray(output_grid)

image_grid

FileNotFoundError: ignored